Нужно собрать выборку разборов

In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
import re

Получаем все ссылки:

In [2]:
page_links = ['http://people.ucalgary.ca/~mmcgilli/OEPoetry/beo-ms.html',
              'http://people.ucalgary.ca/~mmcgilli/OEPoetry/junius.html',
              'http://people.ucalgary.ca/~mmcgilli/OEPoetry/exeter.html',
              'http://people.ucalgary.ca/~mmcgilli/OEPoetry/vercelli.html',
              'http://people.ucalgary.ca/~mmcgilli/OEPoetry/parisps.html',
              'http://people.ucalgary.ca/~mmcgilli/OEPoetry/boethius.html',
              'http://people.ucalgary.ca/~mmcgilli/OEPoetry/minor-poems.html']

def get_links(page_links):
    links = []
    for link in page_links:
        response = requests.get(link)
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        wrong_links = soup.find_all('a', {'class': 'labyrinth'})
        for w_link in  wrong_links:
            links.append('http://people.ucalgary.ca/~mmcgilli' + w_link.attrs['href'][2:])
    return links

all_links = get_links(page_links)

print(len(all_links), 'ссылок нашлось')
print(all_links[100:110])

346 ссылок нашлось
['http://people.ucalgary.ca/~mmcgilli/ASPR/a3.34.4.html', 'http://people.ucalgary.ca/~mmcgilli/ASPR/a3.34.5.html', 'http://people.ucalgary.ca/~mmcgilli/ASPR/a3.34.6.html', 'http://people.ucalgary.ca/~mmcgilli/ASPR/a3.34.7.html', 'http://people.ucalgary.ca/~mmcgilli/ASPR/a3.34.8.html', 'http://people.ucalgary.ca/~mmcgilli/ASPR/a3.34.9.html', 'http://people.ucalgary.ca/~mmcgilli/ASPR/a3.34.10.html', 'http://people.ucalgary.ca/~mmcgilli/ASPR/a3.34.11.html', 'http://people.ucalgary.ca/~mmcgilli/ASPR/a3.34.12.html', 'http://people.ucalgary.ca/~mmcgilli/ASPR/a3.34.13.html']


Функция получения текста произведения по ссылке.

In [3]:
def get_text(link):
    response = requests.get(link)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')
    result = re.search(r'<dl(.|\n)+</dl>', str(soup))
    html_text = result.group(0)
    soup_text = BeautifulSoup(html_text, 'html.parser')
    text = soup_text.text
    return text

Проверка

In [4]:
get_text('http://people.ucalgary.ca/~mmcgilli/ASPR/a3.22.57.html')

'\n\n\nðeos lyft byreð  \xa0 \xa0 \xa0 \xa0  lytle wihte  \nofer beorghleoþa.  \xa0 \xa0 \xa0 \xa0  þa sind blace swiþe,  \nswearte salopade.  \xa0 \xa0 \xa0 \xa0  Sanges rope  \nheapum ferað,  \xa0 \xa0 \xa0 \xa0  hlude cirmað,  \n\n5\n\ntredað bearonæssas,  \xa0 \xa0 \xa0 \xa0  hwilum burgsalo  \nniþþa bearna.  \xa0 \xa0 \xa0 \xa0  Nemnað hy sylfe.  \n\n'

Функция из тетрадки Bede Parser, отправляющая запрос парсеру (но возвращается для удобства только список разборов):

In [5]:
def analyze(word):
    url = 'https://www.bede.net/cgi-bin/parserDublin/parse_dublin.cgi'
    data = {'sen': word}
    html_content = requests.post(url, data).text
    soup = BeautifulSoup(html_content, 'html.parser')
    
    word = soup.find('td', {'valign': 'top'}).text
    a = word.split('\n')
    #print(a)
    word = a[0]
    options = [option[:-2] for option in a[1].split(':')]
    
    final = soup.find('td', {'bgcolor': '#ffff80'}).text
    word_par = {'word': word,
                'options': options,
                'final': final}
    return options

Проверка

In [6]:
print(analyze('god'))

['N S M NOM S', 'N S M ACC S', 'N S N NOM S', 'N S N ACC S', 'N S N NOM PL', 'N S N ACC PL', 'ADJ S M NOM S', 'ADJ S N NOM S', 'ADJ S N ACC S', '']


Параметр для проверки (чтобы цикл шёл не по всем ссылкам, а по одной)

In [7]:
all_links = ['http://people.ucalgary.ca/~mmcgilli/ASPR/a5.59.html']

Проходим по ссылкам, делим текст на слова, для каждого слова добавляем его разборы в список

In [8]:
all_options = []
mess = []
for link in tqdm(all_links):
    text = get_text(link)
    for word in tqdm(text.split()):
        #print(word)
        if word.isalpha():
            opts = analyze(word)
            mess.extend(opts)
            for opt in opts:
                for part in opt.split():
                    if part not in all_options:
                        all_options.append(part)

<ipython-input-8-ab8b5264434e>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for link in tqdm(all_links):


<ipython-input-8-ab8b5264434e>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for word in tqdm(text.split()):


all_options - все уникальные глоссы

mess - все разборы

In [9]:
print(all_options[:5])
print(mess[:5])

['N', 'S', 'F', 'NOM', 'PL']
['N S F NOM S', 'N S N NOM PL', 'N S N ACC PL', 'ADJ S F NOM S', 'ADJ S N NOM PL']


Записываем в текстовый файл

In [10]:
a = ['a', '4', '5']
with open("morf_parts.txt", "w", encoding="utf-8") as f:
    for x in all_options:
        f.write(x)
        f.write('\n')
with open("morf_mess.txt", "w", encoding="utf-8") as f:
    for x in mess:
        f.write(x)
        f.write('\n') 